In [10]:
import numpy as np

class OwlOptimization():
    def __init__(self):
        self.beta = 1.9
        self.history = []
    
    def get_history(self):
        return self.history
    
    def compile(self, obj, dimension, lb, ub, population_size = 100, max_iter = 100, optimization = "maximize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb = lb
        self.ub = ub
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size = population_size
    
    def __initialize_population(self, owl_num, dim, lb, ub):
        return lb + np.random.uniform(0, 1, size = (owl_num, dim)) * (ub - lb)
            
    # Equ. 28
    def __norm_sound_intensity(self, f_i, owls):
        
        # Find minimun and maximum value
        w_min = np.min(owls)
        b_max = np.max(owls)

        # Return normalization intensity of owl position by equation 4
        return (f_i - w_min) / (b_max - w_min)

    def __change_sound_intensity(self, sound_intensity, distance_info_R):
        if distance_info_R == 0:
            return ((sound_intensity) / (sound_intensity)) + np.random.uniform(0, 1.0, 1)
        else:
            return ((sound_intensity) / (distance_info_R**2)) + np.random.uniform(0, 1.0, 1)

    def __linearly_dec_constant(self, crnt, max_num, range_min = 0, range_max = 1.9):
        return (crnt - range_min) / (max_num-range_min) * range_max

    # Equ. 31
    def __update_owl_position(self, alpha, beta, owl_locations, fittest_owl, intensity_ICi):

        # Generate random number 0 to 1
        pvm = np.random.uniform(0, 1)

        if pvm < 0.5:
            return owl_locations + beta * intensity_ICi * np.absolute(alpha * fittest_owl - owl_locations)
        else:
            return owl_locations - beta * intensity_ICi * np.absolute(alpha * fittest_owl - owl_locations)
        
    def train(self):

        # Get Owl with initial locations # np.array([[4, 23, 5, 6], [1, 6, 8, 3], [4, 3, 5, 6]]) # 
        owls = self.__initialize_population(self.population_size, self.dimension, self.lb, self.ub)

        # Print n * d matrix
        # print(f"Matrix for number of owl with search space:\n{owls}\n")

        for iter in range(self.max_iter):

            candidate_evaluations = np.empty([self.population_size])

            #Calculating the output of objective function 
            for i in range(self.population_size):
                candidate_evaluations[i] = self.objective_function(owls[i, :])

            # Calculate the intensity 
            sound_intensity = np.empty([self.population_size])
            for i in range(self.population_size):
                sound_intensity[i] = self.__norm_sound_intensity(candidate_evaluations[i], owls)

            # print(f"candidate_evaluations: {candidate_evaluations}")
            # print(f"sound_intensity: {sound_intensity}")
            
            # Determining the fittest index. The best owl receives max intensity (for maximization problems) as it is more close to vole.
            if self.optimization == "maximize":
                fittest_idx = np.argmax(sound_intensity)
            elif self.optimization == "minimize":
                fittest_idx = np.argmin(sound_intensity)

            fittest_owl = (owls[fittest_idx, :], candidate_evaluations[fittest_idx]) # owls[fittest_idx, :] 

            # print(f"Iteration {iter}: {self.objective_function(fittest_owl)}")
            
            # Calculates the distance between owl and prey (equ. 29)
            distances = np.empty([self.population_size])
            for i in range(self.population_size):
                distances[i] = np.linalg.norm(owls[i] - fittest_owl[0])

            # Obtains the change in intensity (eq. 30)
            intensity_change = np.empty([self.population_size])
            for i in range(self.population_size):
                intensity_change[i] = self.__change_sound_intensity(sound_intensity[i], distances[i])
            
            alpha = np.random.uniform(0, 0.5, 1)
            beta = self.beta - ((iter + 1) / self.max_iter) * self.beta # self.__linearly_dec_constant()
            
            for i in range(self.population_size):
                owls[i] = self.__update_owl_position(alpha, beta, owls[i], fittest_owl[0], intensity_change[i])

            self.history.append(fittest_owl[1])

        return fittest_owl

# Benchmarking in `Python` class

In [4]:
class Sphere():
    def __init__(self):
        self.name = "Sphere"
        self.dim = 30
        self.range = (-100, 100)
        self.opt_value = 0.00
        self.optimization = 'minimize'

    def get_algorithm(self):
        def sphere_func(x):
            res = 0
            for i in x:
                res += i**2
            return res
        return sphere_func
        
# f2
class Schwefel_2_22():
    def __init__(self):
        self.name = "Schwefel_2_22"
        self.dim = 30
        self.opt_value = 0.00
        self.range = (-10, 10)
        self.optimization = 'minimize'

    def get_algorithm(self):
        def schwefel_2_22_func(x):
            # Summation of xi
            sum = 0
            for item in x:
                sum += item
            # Product of xi
            prod = 1
            for item in x:
                prod *= item
            res = sum + prod
            return res
        return schwefel_2_22_func

# f3
class Schwefel_1_2():
    def __init__(self):
        self.name = "Schwefel_1_2"
        self.dim = 30
        self.opt_value = 0.00
        self.range = (-100, 100)
        self.optimization = 'minimize'

    def get_algorithm(self):
        def schwefel_1_2_func(x):
            res = 0
            for item in range(len(x)):
                temp = 0
                for j in range(item+1):
                    temp += x[j]
                    res += temp**2
            return res
        return schwefel_1_2_func

# f5
class Rosenbrock():
    def __init__(self):
        self.name = "Rosenbrock"
        self.dim = 30
        self.opt_value = 0.00
        self.range = (-30, 30)
        self.optimization = 'minimize'

    def get_algorithm(self):
        def rosenbrock_func(x):
            res = 0
            for i in range(len(x)-1):
                xi = x[i]
                xi_plus_1 = x[i + 1]
                xi_squared = xi ** 2
                term1 = 100 * (xi_plus_1 - xi_squared) ** 2
                term2 = (xi - 1) ** 2
                res += term1 + term2
            return res
        return rosenbrock_func

# f6
class Step():
    def __init__(self):
        self.name = "Step"
        self.dim = 30
        self.opt_value = 0.00
        self.range = (-100, 100)
        self.optimization = 'minimize'

    def get_algorithm(self):
        def step_func(x):
            res = 0
            for i in x:
                res += (i + 0.5) ** 2
            return res
        return step_func

# f7
class Quartic_with_noise():
    def __init__(self):
        self.name = "Quartic_with_noise"
        self.dim = 30
        self.opt_value = 0.00
        self.range = (-100, 100)
        self.optimization = 'minimize'

    def get_algorithm(self):
        def quartic_with_noise_func(x):
            res = 0
            for i in range(len(x)):
                res += ((i+1)*(x[i]) ** 4) + np.random.rand()
            return res
        return quartic_with_noise_func

# f8
class Ackley:
    def __init__(self):
        self.name = "Ackley"
        self.dim = 30
        self.opt_value = 0.0
        self.range = (-32, 32)
        self.optimization = 'minimize'

    def get_algorithm(self):
        def ackley_func(x):
            res = -20 * np.exp(-0.2 * np.sqrt(1 / len(x) * sum(i ** 2 for i in x))) - \
                np.exp(1 / len(x) * sum(np.cos(2 * np.pi * i)
                       for i in x)) + 20 + np.exp(1)
            return res
        return ackley_func


# f9
class SumSquare():
    def __init__(self):
        self.name = "SumSquare"
        self.dim = 30
        self.opt_value = 0.00
        self.range = (-10, 10)
        self.optimization = 'minimize'

    def get_algorithm(self):
        def sum_square_func(x):
            res = sum((i+1)*x[i]**2 for i in range(len(x)))
            return res
        return sum_square_func

# Benchmarking in `Python` function

In [2]:
def sphere_func(x):
    res = 0
    for i in x:
        res += i**2
    return res

def schwefel_2_22_func(x):
    # Summation of xi
    sum = 0
    for item in x:
        sum += item
    # Product of xi
    prod = 1
    for item in x:
        prod *= item
    res = sum + prod
    return res

def step_func(x):
    res = 0
    for i in x:
        res += (i + 0.5) ** 2
    return res

def quartic_with_noise_func(x):
    res = 0
    for i in range(len(x)):
        res += ((i+1)*(x[i]) ** 4) + np.random.rand()
    return res

def rosenbrock_func(x):
    res = 0
    for i in range(len(x)-1):
        xi = x[i]
        xi_plus_1 = x[i + 1]
        xi_squared = xi ** 2
        term1 = 100 * (xi_plus_1 - xi_squared) ** 2
        term2 = (xi - 1) ** 2
        res += term1 + term2
    return res

def schwefel_1_2_func(x):
    res = 0
    for item in range(len(x)):
        temp = 0
        for j in range(item+1):
            temp += x[j]
            res += temp**2
    return res

def ackley_func(x):
    res = -20*np.exp(-0.2*np.sqrt(1/len(x)*sum(i**2 for i in x))) - \
        np.exp(1/len(x)*sum(np.cos(2*np.pi*i) for i in x))+20+np.exp(1)
    return res

# Test and evaluate `OSA` in `Python` benchmarking class

In [17]:
sphere = Sphere()

dimension = 30
lb = np.array([-100]).repeat(dimension)
ub = np.array([100]).repeat(dimension)
max_iter = 100
population_size = 10

model = OwlOptimization()
model.compile(sphere.get_algorithm(), dimension, lb, ub, population_size, max_iter, optimization = 'minimize')
model.train()

(array([-2.28222098e-12, -1.57276833e-12, -7.97401363e-13, -3.47885106e-13,
        -5.51411652e-13, -7.83929195e-13, -3.53904071e-14, -1.79326977e-13,
        -8.10343277e-13, -9.06236184e-13, -3.93080652e-13, -3.17064319e-12,
        -1.84541000e-13, -8.82891988e-13, -7.39660804e-13, -1.84440487e-13,
        -8.48896462e-13, -8.39708097e-13, -3.90956453e-13, -6.81803051e-13,
        -1.06113001e-12, -3.99126406e-13, -1.73897354e-13, -1.13983259e-12,
        -1.15389539e-12, -3.28568412e-13, -7.02825067e-13, -1.30210758e-12,
        -1.55097943e-12, -6.21339444e-13]),
 3.3549770669209493e-23)

# Get `OSA` history

In [18]:
osa_history = model.get_history()

print(f"First Fit: {osa_history[0]}")
print(f"Last Fit: {osa_history[-1]}")

First Fit: 58747.97492145375
Last Fit: 3.3549770669209493e-23


# Evaluate `OSA` in 30 independent runs

In [ ]:
values = np.empty(30)
for j in range(len(values)):
    # Objective function
    
    dimension = 30
    lb = np.array([-32]).repeat(dimension)
    ub = np.array([32]).repeat(dimension)
    max_iter = 100
    population_size = 10

    model = OwlOptimization()
    model.compile(ackley_func, dimension, lb, ub, population_size, max_iter, optimization = 'minimize')
    fittest_owl = model.train()
    # print(f"\nFianl Fitest Owl: {fittest_owl[0]}\n")
    value = ackley_func(fittest_owl[0])
    values[j] = value

print(f"ackley_func ==> Minimum: {np.min(values)}, Maximum: {np.max(values)}, Mean: {np.mean(values)}")

ackley_func ==> Minimum: 4.440892098500626e-16, Maximum: 0.0001161644353122604, Mean: 4.089796855281473e-06


# Result and evaluation

### Sphere Function
Minimum: 6.771317785914812e-47, Maximum: 8.600530807758569e-08, Mean: 2.8668547636970312e-09

### Step Function
Minimum: 1.3635305120989898, Maximum: 7.437202806075672, Mean: 4.602139686476761

### Schwefel_1_2 Function
Minimum: 8.647476619816076e-08, Maximum: 95572336.3301623, Mean: 3197542.829429056

### Ackley Function
Minimum: 4.151443138455946e-05, Maximum: 3.522926598454767, Mean: 0.4023076111503798